# 1. Importación de librerias necesarias

In [2]:
import pandas as pd 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import plotly.express as px
import streamlit as st
from dotenv import load_dotenv
import os

# 2. Leyendo credenciales del API de Spotify

In [3]:
load_dotenv()  # Carga las variables desde el archivo .env

client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')

In [4]:
# Autenticacióin del usuario
try: 
    # Capturando las credenciales del usuario
    client_credentials_manager = SpotifyClientCredentials(client_id= client_id, client_secret=client_secret)
    
    #Guardando la conección de las credenciales
    spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
    print("Conexión exitosa")

except Exception as e:
    print(f"Hubo un error al ejecutar la autenticación de las credenciales: {e}")

Conexión exitosa


# 3. Creando funciones necesarias

In [19]:
def DataframeArtistaSpotify(sp, artista):
    """
    Crea un DataFrame de Pandas con información detallada sobre las canciones de un artista en Spotify.

    Args:
        sp: Objeto de conexión a la API de Spotify.
        artista (str): Nombre del artista a buscar.

    Returns:
        pd.DataFrame: DataFrame con las siguientes columnas:
            - Álbum: Nombre del álbum.
            - Tipo: Tipo de lanzamiento (Álbum o Single).
            - Año: Año de lanzamiento del álbum.
            - Canción: Título de la canción.
            - Artistas: Lista de artistas de la canción.
            - Duración: Duración de la canción en formato minutos:segundos.
            - Popularidad: Popularidad de la canción en una escala del 0 al 100.

    Esta función realiza los siguientes pasos:
        1. Busca el ID del artista en la API de Spotify utilizando el nombre proporcionado.
        2. Obtiene los álbumes y singles del artista.
        3. Itera sobre cada álbum y single, extrayendo información sobre cada canción.
        4. Crea una lista con los datos de todas las canciones.
        5. Convierte la lista en un DataFrame de Pandas.
    """
    
    # Buscamos el ID del artista en la API de Spotify
    resultados = sp.search(q=artista, limit=1, type='artist')
    artista_id =  resultados['artists']['items'][0]['id']

    # Crear una lista vacía para almacenar los datos de las canciones
    canciones_data = [] 
    
    # Obtenemos los álbumes y singles del artista
    albumes = sp.artist_albums(artista_id) # Obtener los álbumes del artista

    # Iteramos sobre cada álbum y single
    for album in albumes['items']: # Obtener las canciones para cada álbum
        # Extraemos información del álbum
        album_nombre = album['name']
        album_tipo = album['album_type']
        album_año = album['release_date'].split('-')[0]  # Obtener el año de lanzamiento
        
        # Obtenemos las pistas del álbum
        tracks = sp.album_tracks(album['id'])

        # Iteramos sobre cada canción
        for track in tracks['items']: # Para cada canción, obtener los detalles y añadirlos a la lista de canciones
            cancion_nombre = track['name']
            cancion_artistas = ', '.join([t['name'] for t in track['artists']])
            cancion_duracion = '{:02d}:{:02d}'.format(*divmod(track['duration_ms'] // 1000, 60))
            cancion_popularidad = sp.track(track['id'])['popularity'] # Obtener la popularidad de la canción
            
            # Añadir los detalles de cadaa canción a la lista de canciones
            canciones_data.append([album_nombre, album_tipo, album_año, cancion_nombre, cancion_artistas, cancion_duracion, cancion_popularidad])
    
    return pd.DataFrame(canciones_data, columns=['Álbum', 'Tipo', 'Año', 'Canción', 'Artistas', 'Duración', 'Popularidad'])
    

In [27]:
def crear_grafico_popularidad(df):
    """
    Crea un gráfico de barras de las canciones más populares.

    Args:
        df (pd.DataFrame): DataFrame con los datos de las canciones.

    Returns:
        plotly.graph_objects.Figure: Figura del gráfico.
    """

    canciones_populares = df.sort_values(by=['Popularidad'], ascending=True).head(20)
    fig = px.bar(canciones_populares, 
                x='Canción', 
                y='Popularidad',
                text='Popularidad',
                color="Tipo",
                hover_data={'Álbum':True, 'Año':True},
                title='Canciones más populares',
                template='plotly_dark')

    # Personalización del gráfico
    fig.update_layout(
        title={'text': '<i>Canciones más populares</i>',
            'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
            'font':dict(size=25, color='white')},
        xaxis=dict(categoryorder='total descending', title_font=dict(size=18), tickfont=dict(size=18)),
        yaxis=dict(range=[0, canciones_populares['Popularidad'].max()+5],title_font=dict(size=18), showgrid=False, showticklabels=False)
    )
    
    fig.update_traces(textposition='outside', textfont_size=18)

    return fig


# 4. Creando el dataframe

In [20]:
data = DataframeArtistaSpotify(spotify, "Lucho Quequezana")

data

,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad
0,Pangea,album,2018,Mayu,Lucho Quequezana,03:17,18
1,Pangea,album,2018,Renacer,Lucho Quequezana,02:38,15
2,Pangea,album,2018,La Cumbia De Don Carpio,Lucho Quequezana,04:21,18
3,Pangea,album,2018,Pangea,Lucho Quequezana,03:46,13
4,Pangea,album,2018,Facu,Lucho Quequezana,02:46,17
5,Pangea,album,2018,Otorongo,Lucho Quequezana,05:34,14
6,Pangea,album,2018,El Caimán,Lucho Quequezana,06:07,13
7,Pangea,album,2018,Mujeres,Lucho Quequezana,03:01,18
8,Pangea,album,2018,Mantaro,Lucho Quequezana,04:07,20
9,Combi,album,2014,En Primavera,Lucho Quequezana,03:35,15


In [21]:
data["Popularidad"].unique()

array([18, 15, 13, 17, 14, 20, 16, 23, 12, 27, 25, 21, 31, 26, 24, 19, 37,
       48, 46, 11, 10, 32, 36, 33,  1,  2,  0,  8,  5], dtype=int64)

# 5. Creando graficos necesarios

In [28]:
crear_grafico_popularidad(data)

# 6. Creando código de Streamlit

In [29]:
def main():
    st.title('Explorador de Música de Spotify')

    # Input del usuario para el nombre del artista
    artista = st.text_input('Ingrese el nombre del artista:')

    # Botón para buscar
    if st.button('Buscar'):
        # Lógica para obtener los datos
        df = DataframeArtistaSpotify(spotify, artista)

        # Visualización de los datos (ejemplo con Plotly)
        fig = crear_grafico_popularidad(df)
        st.plotly_chart(fig)


if __name__ == '__main__':
    main()


2024-08-11 16:17:21.326 
  command:

    streamlit run C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-08-11 16:17:21.351 Session state does not function when running a script without `streamlit run`
